In [1]:
%load_ext autoreload
%autoreload 2    

In [26]:
import csv
from torchvision import datasets, models, transforms
import numpy as np
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torchvision

from augmentation import *
from train import train
from config.transfer_config import *

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
transforms_0

Compose(
    ToTensor()
    Resize(size=(300, 300), interpolation=bilinear, max_size=None, antialias=True)
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)

In [5]:
trainset = datasets.ImageFolder('/dtu/datasets1/02514/hotdog_nothotdog/train', transform=transforms_0)
train_loader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8, pin_memory=True, persistent_workers=True)

testset = datasets.ImageFolder('/dtu/datasets1/02514/hotdog_nothotdog/test', transform=transforms_0)
test_loader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8, pin_memory=True, persistent_workers=True)

In [6]:
torch.cuda.empty_cache()

# Options

1. training only the head with normal learning rate
2. training all with super low training rate

In [37]:
def transfer_model_set(model, freeze_convs=False,):
    
    if freeze_convs:
        print('Freezing Convs')
        # freeze the feature extractors
        for param in model.parameters():
            param.requires_grad = False
    
    if type(model) == torchvision.models.densenet.DenseNet:
        in_features = model.classifier.in_features
    
    elif type(model) == torchvision.models.resnet.ResNet:
        in_features = model.fc.in_features
    
    
    size_hidden = 512
    out_features = 10
    
    head = nn.Sequential(
                    nn.Linear(in_features, size_hidden),
                    nn.Dropout(DROP_OUT_RATE),
                    nn.ReLU(),
                    nn.BatchNorm1d(size_hidden),
                    nn.Linear(size_hidden, out_features))
    
    if type(model) == torchvision.models.densenet.DenseNet:
        model.classifier = head
    
    elif type(model) == torchvision.models.resnet.ResNet:
        model.fc = head

    else:
        raise Exception('Not implemented the classifier for this type of model')

    model = model.to(device)

    return model

# 1. Training just head

In [8]:
model = models.resnet50(weights=torchvision.models.ResNet50_Weights.IMAGENET1K_V2) # lets take v2 here 

model = transfer_model_set(model, freeze_convs=True)

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=HEAD_LEARNING_RATE)

out_dict_resnet50 = train(model, train_loader, test_loader, loss, optimizer, NUM_EPOCHS)

# saving results
optim = 'Adam'
d = out_dict_resnet50
with open(f'results/results_transfer_resnet_head_{NUM_EPOCHS}_epochs_{HEAD_LEARNING_RATE:.0e}_lr_{optim}_optim.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(d.keys())
    writer.writerows(zip(*d.values()))

In [38]:
model = models.densenet121(weights=torchvision.models.DenseNet121_Weights.IMAGENET1K_V1)

model = transfer_model_set(model, freeze_convs=True)

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=HEAD_LEARNING_RATE)

out_dict = train(model, train_loader, test_loader, loss, optimizer, NUM_EPOCHS)

# saving results
optim = 'Adam'
d = out_dict
with open(f'results/results_transfer_densenet_head_{NUM_EPOCHS}_epochs_{HEAD_LEARNING_RATE:.0e}_lr_{optim}_optim.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(d.keys())
    writer.writerows(zip(*d.values()))

Freezing Convs


  0%|          | 0/20 [00:00<?, ?epoch/s]

  0%|          | 0/32 [00:00<?, ?it/s]

Loss train: 0.680	 test: 0.281	 Accuracy train: 86.2%	 test: 93.7%


  0%|          | 0/32 [00:00<?, ?it/s]

Exception in thread Thread-6 (_pin_memory_loop):
Traceback (most recent call last):
  File "/appl/python/3.10.11/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/appl/python/3.10.11/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/zhome/8d/e/198218/dlincv/lib/python3.10/site-packages/torch/utils/data/_utils/pin_memory.py", line 51, in _pin_memory_loop
    do_one_step()
  File "/zhome/8d/e/198218/dlincv/lib/python3.10/site-packages/torch/utils/data/_utils/pin_memory.py", line 28, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/appl/python/3.10.11/lib/python3.10/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
  File "/zhome/8d/e/198218/dlincv/lib/python3.10/site-packages/torch/multiprocessing/reductions.py", line 307, in rebuild_storage_fd


KeyboardInterrupt: 

    fd = df.detach()
  File "/appl/python/3.10.11/lib/python3.10/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/appl/python/3.10.11/lib/python3.10/multiprocessing/resource_sharer.py", line 86, in get_connection
    c = Client(address, authkey=process.current_process().authkey)
  File "/appl/python/3.10.11/lib/python3.10/multiprocessing/connection.py", line 502, in Client
    c = SocketClient(address)
  File "/appl/python/3.10.11/lib/python3.10/multiprocessing/connection.py", line 630, in SocketClient
    s.connect(address)
FileNotFoundError: [Errno 2] No such file or directory


# Train everything (but slowly! 🐢🐢🐢)

In [39]:
model = models.resnet50(weights=torchvision.models.ResNet50_Weights.IMAGENET1K_V2) # lets take v2 here 

model = transfer_model_set(model, freeze_convs=False)

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=HEAD_LEARNING_RATE)

out_dict_resnet50 = train(model, train_loader, test_loader, loss, optimizer, NUM_EPOCHS)

# saving results
optim = 'Adam'
d = out_dict_resnet50
with open(f'results/results_transfer_resnet_full_{NUM_EPOCHS}_epochs_{HEAD_LEARNING_RATE:.0e}_lr_{optim}_optim.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(d.keys())
    writer.writerows(zip(*d.values()))

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /zhome/8d/e/198218/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|████████████████████████████████████████████████| 97.8M/97.8M [00:01<00:00, 85.9MB/s]


  0%|          | 0/20 [00:00<?, ?epoch/s]

RuntimeError: Pin memory thread exited unexpectedly

In [ ]:
model = models.densenet121(weights=torchvision.models.DenseNet121_Weights.IMAGENET1K_V1)

model = transfer_model_set(model, freeze_convs=False)

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=HEAD_LEARNING_RATE)

out_dict = train(model, train_loader, test_loader, loss, optimizer, NUM_EPOCHS)

# saving results
optim = 'Adam'
d = out_dict
with open(f'results/results_transfer_densenet_full_{NUM_EPOCHS}_epochs_{HEAD_LEARNING_RATE:.0e}_lr_{optim}_optim.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(d.keys())
    writer.writerows(zip(*d.values()))